In [ ]:
import numpy as np
import time
from importlib import reload
import nbimporter
import obebs_func
reload(obebs_func)

###############################################################################################################################
# load MNIST data 
###############################################################################################################################

cnum = 10
cluster_centers = np.loadtxt('obebs_data/cluster_centers.txt', dtype=np.float64)
Q = np.loadtxt('obebs_data/Q.txt', dtype=np.float64)
labels = np.loadtxt('obebs_data/labels.txt', dtype=np.int32)
X_pool = np.loadtxt('obebs_data/X_pool.txt', dtype=np.float64)
Y_pool = np.loadtxt('obebs_data/Y_pool.txt', dtype=np.int32)

###############################################################################################################################
# RUN OBEBS
###############################################################################################################################

iternum = 100
tt = time.time()
X_train = []
Y_train = []
labels_train = []
accuracy = []
entropy_list = []
optimal_entropy_list = []
Diff_t = 0
LBE = []
adaptive_reset = 0
adaptive_threshold = 3
MID = None
for i in range(iternum):
    TP = FP = FN = TN = 0
    predictions = []
    divide_flag = False
    adaptive_reset -= 1
    if (adaptive_reset < 1) and (len(X_train) > 20):
        for iii in range(cnum-1):
            for jjj in range(iii+1,cnum):
                if abs(X_train[Y_train == iii,:].shape[0] - X_train[Y_train == jjj,:].shape[0]) > adaptive_threshold:
                    divide_flag = True
                    break
        if divide_flag:
            print('Running adaptive assignment!')
            buckets = np.zeros(cnum)
            for iii in range(cnum):
                buckets[iii] = X_train[Y_train == iii,:].shape[0]
            labels,labels_train,Q,MID = obebs_func.adaptive_assignment(cnum,labels,labels_train,X_pool,X_train,Y_train,Q,costMX,MID)
            adaptive_reset = cnum*(adaptive_threshold-1)
    if len(X_train) > 0:
        costMX = obebs_func.get_costMX(Y_train, labels_train, cnum)
        A = obebs_func.hun_algo(costMX)
        P = np.matmul(Q,A)
    else:
        P = Q
    for j in range(P.shape[0]):
        predictions.append(np.argmax(P[j,:]))
    accuracy.append(sum(predictions==Y_pool)/len(Y_pool))
    X_train,Y_train,X_pool,Y_pool,labels,labels_train,maxIdx,metric = obebs_func.OBEBS(X_train,Y_train,X_pool,Y_pool,P,labels,labels_train,cnum)
    Q = np.delete(Q, maxIdx, axis=0)
    Diff_t += obebs_func.Diff_t_(cnum, X_train, Y_train)
    LBE.append((Diff_t - (i+1) + int((i+1)/cnum))/(i+1))
    entropy_list.append(obebs_func.calculate_entropy(X_train, Y_train, cnum))
    optimal_entropy_list.append(obebs_func.calculate_optimal_entropy(X_train, Y_train, cnum))
    print(i, end = ' ')
print('Test time: ', time.time()-tt)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.hist(Y_train+1) # Y_train = list of true labels of the queried items
plt.title("Bucket distribution") 
plt.show()